## Задание 2

Состоит из **обязательной** и **бонусной** частей.

Обязательная часть оценивается в **50 баллов** и выполняется до **16 декабря 09:00**.

Бонусную часть можно делать, пока не придет необходимость получения оценки/зачета.

Обязательная часть заключается в fine-tuning несложной нейросети (ResNet-18) на UIUC Sports Event Dataset (http://vision.stanford.edu/lijiali/event_dataset) и ее последующем ускорении с помощью фреймворка **NVIDIA TensorRT** (https://developer.nvidia.com/tensorrt).

In [0]:
import torch
import torchvision
from torchvision import transforms, models

from tqdm import tqdm_notebook as tqdm

**(5 баллов)** Скачайте датасет, распакуйте его в директорию `./event_img/`. В ней должны оказаться 8 директорий, соответствующих классам картинок. Загрузите датасет в torch и разбейте случайным образом на train и val.

In [0]:
#!wget http://vision.stanford.edu/lijiali/event_dataset/event_dataset.rar
#!unrar x event_dataset.rar

In [3]:
event_dataset = torchvision.datasets.ImageFolder('event_img')
assert isinstance(event_dataset, torch.utils.data.Dataset)

dataset_length = len(event_dataset)
print(f'Number of images: {dataset_length}')

Number of images: 1579


In [0]:
val_size = .4
val_length = int(dataset_length * val_size)
train_length = dataset_length - val_length

train_data_raw, val_data_raw = torch.utils.data.random_split(event_dataset, [train_length, val_length])

assert isinstance(train_data_raw, torch.utils.data.Dataset)
assert isinstance(val_data_raw, torch.utils.data.Dataset)

**(10 баллов)** Нам нужны разные преобразования (transforms) для train и val. Напишите класс `ApplyTransform`, объект которого — тот же датасет, что подается в конструкторе, но с примененными преобразованиями.

К `train_data_raw` нужно применить изменение размера до 256px (`min(height, width)`), извлечение региона 256x256 в центре, выбор в этом регионе случайного квадрата 224x224.

К `test_data_raw` нужно применить изменение размера до 224px и извлечение региона 224x224 в центре.

In [0]:
class ApplyTransform(torch.utils.data.Dataset):
    def __init__(self, dataset, transform=None, target_transform=None):
        self.data = dataset
        self.transform = transform

    def __len__(self):
        return self.data.__len__()

    def __getitem__(self, index):
        if self.transform:
            X, y = self.data.__getitem__(index)
            return self.transform(X), y
        return self.data.__getitem__(index)

In [0]:
imagenet_normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

train_transform = transforms.Compose([
    torchvision.transforms.Resize(256),
    torchvision.transforms.CenterCrop(256),
    torchvision.transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    imagenet_normalize
])

val_transform = transforms.Compose([
    torchvision.transforms.Resize(224),
    torchvision.transforms.CenterCrop(224),
    transforms.ToTensor(),
    imagenet_normalize
])

train_data = ApplyTransform(train_data_raw, train_transform)
val_data = ApplyTransform(val_data_raw, val_transform)

In [0]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=8)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, num_workers=8)

**(5 баллов)** Загрузите предобученную на ImageNet модель ResNet-18, адаптируйте ее под классификацию на 8 классов. Создайте подходящую функцию потерь и оптимизатор SGD с `momentum=0.9`.

In [0]:
num_classes = 8

model = torchvision.models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, num_classes)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

**(10 баллов)** Реализуйте обучение модели, а потом измерьте время инференса на val датасете (`batch_size=32`).

In [9]:
num_epochs = 10

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

for epoch in tqdm(range(num_epochs)):
    print("Epoch {:>2}/{:>2}".format(epoch + 1, num_epochs), end='')
    for X, y in train_loader:
        print('.', end='')

        X = X.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        model.train()

        preds = model.forward(X)

        loss_value = criterion(preds, y)
        loss_value.backward()

        optimizer.step()
    print()


Epoch  1/10..............................
Epoch  2/10..............................
Epoch  3/10..............................
Epoch  4/10..............................
Epoch  5/10..............................
Epoch  6/10..............................
Epoch  7/10..............................
Epoch  8/10..............................
Epoch  9/10..............................
Epoch 10/10..............................



In [10]:
import time
import numpy as np

t = []
model.eval()
for X, y in val_loader:
    X = X.to(device)
    y = y.to(device)

    start = time.monotonic()
    model.forward(X)
    t.append(time.monotonic() - start)

print(np.mean(t))

0.012406075849867193


In [0]:
# measure inference time
# 0.012406075849867193

**(20 баллов)** Установите TensorRT и **torch2trt** (https://github.com/NVIDIA-AI-IOT/torch2trt). Оптимизируйте с помощью torch2trt обученную модель и снова измерьте время инференса.

Попробуйте:
* как режим fp32, так и fp16
* не менее трех разных значений `batch_size`

In [0]:
from torch2trt import torch2trt

In [13]:
for batch_size in [ 1, 32, 128, 512 ]:
    for fp16_mode in [ False, True ]:
        try:
            model_trt = torch2trt(model, [[X.to(device) for X, y in train_loader][0]], max_batch_size=batch_size, fp16_mode=fp16_mode)
            t = []
            for X, y in val_loader:
                X = X.to(device)
                y = y.to(device)

                start = time.monotonic()
                model_trt.forward(X)
                t.append(time.monotonic() - start)

            print("batch_size {:>3}, fp16_mode {:5}: {}".format(batch_size, str(fp16_mode), np.mean(t)))
        except Exception:
            pass


batch_size   1, fp16_mode False: 0.001258589000281063
batch_size   1, fp16_mode True : 0.0022488963995783706
batch_size  32, fp16_mode False: 0.002087083900551079
batch_size  32, fp16_mode True : 0.0018581229998744675
batch_size 128, fp16_mode False: 0.0031716563496956953
batch_size 128, fp16_mode True : 0.0020114141503654536
batch_size 512, fp16_mode False: 0.002283826250277343
batch_size 512, fp16_mode True : 0.003457137350051198


In [0]:
# measure inference time
# 0.001258589000281063
#
#   Лучшее время достагается при размере батча 1 и отключённом режиме fp16_mode (параметры по умолчанию).
#   По сравнению с pytorch-реализацией инференс ускоряется почти в 10 раз.

### Бонусная часть (100 баллов)
Нужно переписать функцию `torch2trt()`, чтобы она поддерживала **режим `int8`-инференса**.

Сам TensorRT в режиме `int8` требует **калибровки**, которую нужно выполнять по train датасету. То есть нужно написать класс-калибратор (подкласс `tensorrt.IInt8EntropyCalibrator2`).

За уточнением того, что надо сделать, и за помощью обращаться к:
* https://docs.nvidia.com/deeplearning/sdk/tensorrt-developer-guide/index.html#enable_int8_python
* https://docs.nvidia.com/deeplearning/sdk/tensorrt-api/python_api/index.html
* примеру из TensorRT `samples/python/int8_caffe_mnist`
* преподавателю через почтовый ящик курса или telegram